In [20]:
import lancedb
import pandas as pd 
import numpy as np 
import os 
import ast
from sentence_transformers import SentenceTransformer

In [3]:
data_dir = "../../europeancities-wikivoyage-tripadvisor/data-sources/"
wikivoyage_docs_dir = data_dir + "wikivoyage/"
wikivoyage_listings_dir = wikivoyage_docs_dir + "listings/"
airbnb_dir = data_dir + "airbnb/"

In [25]:
uri = "../database/wikivoyage-listings-db"
db = lancedb.connect(uri)

In [9]:
listings = pd.read_csv(wikivoyage_listings_dir + "wikivoyage-listings-embeddings.csv")

In [12]:
listings = listings[['city','type','title','description','country','combined', 'embeddings']]

In [13]:
listings.rename(columns={'embeddings':'vector'}, inplace=True)


/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_68843/3595120442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings.rename(columns={'embeddings':'vector'}, inplace=True)


In [14]:
listings.head()

,city,type,title,description,country,combined,vector
0,Aalborg,see,Aalborg Akvavit,Distillery.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.03534001111984253, 0.04959840700030327, -0...."
1,Aalborg,see,Aalborghus Castle,A castle and garden from the 16th century.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.020535504445433617, 0.10963066667318344, -0..."
2,Aalborg,see,Aalborg Tower,No description found.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.0070536076091229916, 0.10140001028776169, -..."
3,Aalborg,see,Aalborg Townhall,Built in 1759.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[-0.030464187264442444, 0.050615373998880386, ..."
4,Aalborg,see,Aalborg Zoo,"Cosy zoo with many ""classic"" zoo animals such ...",Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.09230892360210419, 0.08421209454536438, -0...."


In [17]:
listings['vector'] = listings['vector'].apply(ast.literal_eval)

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_68843/3399729079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['vector'] = listings['vector'].apply(ast.literal_eval)


In [18]:
listings.head()

,city,type,title,description,country,combined,vector
0,Aalborg,see,Aalborg Akvavit,Distillery.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.03534001111984253, 0.04959840700030327, -0...."
1,Aalborg,see,Aalborghus Castle,A castle and garden from the 16th century.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.020535504445433617, 0.10963066667318344, -0..."
2,Aalborg,see,Aalborg Tower,No description found.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.0070536076091229916, 0.10140001028776169, -..."
3,Aalborg,see,Aalborg Townhall,Built in 1759.,Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[-0.030464187264442444, 0.050615373998880386, ..."
4,Aalborg,see,Aalborg Zoo,"Cosy zoo with many ""classic"" zoo animals such ...",Denmark,"city: Aalborg, country: Denmark, type: see, ti...","[0.09230892360210419, 0.08421209454536438, -0...."


In [19]:
db.create_table("wikivoyage_listings", data=listings)

LanceTable(connection=LanceDBConnection(/Users/adithisatish/TUM/Academics/SoSe24/guided_research/rag-sustainable-trs/vectordb/../database/wikivoyage-listings-db), name="wikivoyage_listings")

In [26]:
def embed_query(query):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # vector_dimension = model.get_sentence_embedding_dimension()   
    embedding = model.encode(query).tolist()
    return embedding

query = "Museums in Europe"
query_embedding = embed_query(query)
table = db.open_table("wikivoyage_listings")
results = table.search(query_embedding).limit(5).to_list()


In [33]:
[r["combined"] for r in results]

["city: Bologna, country: Italy, type: see, title: Museo Europeo degli Studenti, description: The ''Museum of European Students'' is about the history and culture of university students from the thirteenth to the twentieth century. Closed on Monday; Admission free. Don't miss this when you are interested in student life. It's unique.",
 'city: Stuttgart, country: Germany, type: see, title: Staatliches Museum für Naturkunde Stuttgart, description: Two exhibitions: one features biology and evolution and the other features paleontology and archeology.',
 'city: Lyon, country: France, type: see, title: Musée gallo-romain de Fourvière, description: The second largest museum in France, it has all kinds of things relating to Rhone-Alps history. A free visit to the Roman theatres may be just as interesting for those not into the details.',
 'city: Strasbourg, country: France, type: see, title: Musée Alsacien, description: This museum features articles from the daily lives of Alsatian peoples f

In [34]:
[f"city: {r['city']}, country: {r['country']}, name: {r['title']}, description: {r['description']}" for r in results]

["city: Bologna, country: Italy, name: Museo Europeo degli Studenti, description: The ''Museum of European Students'' is about the history and culture of university students from the thirteenth to the twentieth century. Closed on Monday; Admission free. Don't miss this when you are interested in student life. It's unique.",
 'city: Stuttgart, country: Germany, name: Staatliches Museum für Naturkunde Stuttgart, description: Two exhibitions: one features biology and evolution and the other features paleontology and archeology.',
 'city: Lyon, country: France, name: Musée gallo-romain de Fourvière, description: The second largest museum in France, it has all kinds of things relating to Rhone-Alps history. A free visit to the Roman theatres may be just as interesting for those not into the details.',
 'city: Strasbourg, country: France, name: Musée Alsacien, description: This museum features articles from the daily lives of Alsatian peoples from the 13th to 19th centuries: clothing, furnit